# WALMART PROJET

In [32]:
# libraries import

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib.widgets import Lasso
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score
from datetime import datetime
from sklearn.linear_model import LinearRegression
import warnings
import plotly.express as px
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

In [33]:
# reading the dataset
df = pd.read_csv('Walmart_store_sales.csv')

In [34]:
# Basic stats
print("Number of rows : {}".format(df.shape[0]))
print("Number of columns : {}".format(len(df.columns)))
print()

print("Display of dataset: ")
display(df.head())
print()

print("Basics statistics: ")
data_desc = df.describe(include='all') # include all, donne des info aussi  des sur les catégoriels 
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Number of rows : 150
Number of columns : 8

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

 ## Preprocessing 
 ### Pandas


In [35]:
# drop missing value in the target variable (Y) Weekly_Sales. 
mask = df['Weekly_Sales'].isnull() != True
df2 = df[mask]
df2.shape

(136, 8)

In [36]:
# transform date to be usable by our model 

df2['Date'] = pd.to_datetime(df2['Date'], format='%d-%m-%Y')

df2['day'] = df2['Date'].dt.day
df2['month'] = df2['Date'].dt.month 
df2['year'] =  df2['Date'].dt.year
df2['dayofweek'] = df2['Date'].dt.dayofweek

df2


/var/folders/38/qp_mmpqs5_5fc3fgl0jc4_v40000gn/T/ipykernel_18490/3123502291.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/38/qp_mmpqs5_5fc3fgl0jc4_v40000gn/T/ipykernel_18490/3123502291.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/38/qp_mmpqs5_5fc3fgl0jc4_v40000gn/T/ipykernel_18490/3123502291.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,day,month,year,dayofweek
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858,18.0,2.0,2011.0,4.0
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470,25.0,3.0,2011.0,4.0
3,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092,28.0,5.0,2010.0,4.0
5,4.0,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896,28.0,5.0,2010.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
145,14.0,2010-06-18,2248645.59,0.0,72.62,2.780,182.442420,8.899,18.0,6.0,2010.0,4.0
146,7.0,NaT,716388.81,NaN,20.74,2.778,NaN,NaN,NaN,NaN,NaN,NaN
147,17.0,2010-06-11,845252.21,0.0,57.14,2.841,126.111903,NaN,11.0,6.0,2010.0,4.0
148,8.0,2011-08-12,856796.10,0.0,86.05,3.638,219.007525,NaN,12.0,8.0,2011.0,4.0


In [37]:
# All days are friday, we can know deliete this columns and the Date one :
del df2['Date']
del df2['dayofweek']

**Drop lines containing invalid values or outliers :**
In this project, will be considered as outliers all the numeric features that don't fall within the range : $[\bar{X} - 3\sigma, \bar{X} + 3\sigma]$. This concerns the columns : *Temperature*, *Fuel_price*, *CPI* and *Unemployment*

In [38]:
# we don't see invalid value in the data up. No negative values, degrees are logical (between -7 and 33°)

In [39]:
# Outliers

columns_list = ['Temperature', 'Fuel_Price', 'CPI' ,'Unemployment']

def compute_mask_exclude_outliers(dataframe, column):
    return ((dataframe[column] > (dataframe[column].mean() - ( 3 * dataframe[column].std()))) \
                  & (dataframe[column] < (dataframe[column].mean() + ( 3 * dataframe[column].std())))
            ) \
            | (dataframe[column].isnull())
 

mask = pd.Series(True, index=df2.index)
for column in columns_list:
    mask = mask & compute_mask_exclude_outliers(df2, column)



In [40]:
df_clean = df2[mask]
df_clean.shape

(131, 10)

In [41]:
# Separate target variable Y from features X
target_name = 'Weekly_Sales'

print("Separating labels from features...")
Y = df_clean.loc[:,target_name]
X = df_clean.drop(target_name, axis = 1) 
print("...Done.")
print(Y.head())
print()
print(X.head())
print()

Separating labels from features...
...Done.
0    1572117.54
1    1807545.43
3    1244390.03
4    1644470.66
5    1857533.70
Name: Weekly_Sales, dtype: float64

   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
0    6.0           NaN        59.61       3.045  214.777523         6.858   
1   13.0           0.0        42.38       3.435  128.616064         7.470   
3   11.0           0.0        84.57         NaN  214.556497         7.346   
4    6.0           0.0        78.89       2.759  212.412888         7.092   
5    4.0           0.0          NaN       2.756  126.160226         7.896   

    day  month    year  
0  18.0    2.0  2011.0  
1  25.0    3.0  2011.0  
3   NaN    NaN     NaN  
4  28.0    5.0  2010.0  
5  28.0    5.0  2010.0  



 ## Preprocessing - Scikit-Learn

In [42]:
# Divide dataset into train set & test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



### Preprocessings : imputation of missing values, standardizing and one-hot encoding

In [43]:
# Create pipeline for numeric features
numeric_features = ['Temperature', 'Fuel_Price',
       'CPI', 'Unemployment', 'day', 'month', 'year'] 
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # missing values will be replaced by columns' median (it could be mean, the 2 are really close in our dataset)
    ('scaler', StandardScaler())
])

In [44]:
# Create pipeline for categorical features
categorical_features = ['Store','Holiday_Flag'] # Names of categorical columns in X_train/X_test
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [45]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [46]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5]) # MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head()) 
X_test = preprocessor.transform(X_test) # Don't fit to don't biased
print(X_test[0:5,:]) # MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
print()

Performing preprocessings on train set...
     Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
136    4.0           NaN        84.59       3.469  129.112500         5.644   
78     1.0           0.0        62.25       3.308  218.220509         7.866   
17    18.0           0.0        21.33       2.788  131.527903         9.202   
108   18.0           0.0        69.12       2.906  132.293936           NaN   
141    5.0           0.0        62.37         NaN  212.560411         6.768   

      day  month    year  
136   8.0    7.0  2011.0  
78   18.0   11.0  2011.0  
17    NaN    NaN     NaN  
108  28.0    5.0  2010.0  
141  12.0   11.0  2010.0  
...Done.
[[ 1.46315567  0.30806957 -1.35931117 -1.84213463 -1.16921154  0.22259605
   0.17418541  0.          0.          1.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.13891797

### Part 2 : Baseline model (linear regression)

In [47]:
# Train model
regressor = LinearRegression()

print("Training model...")
regressor.fit(X_train, Y_train) 
print("...Done.")

Training model...
...Done.


In [48]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = regressor.predict(X_train)
print("...Done.")

# Predictions on test set
print("Predictions on test set...")
Y_test_pred = regressor.predict(X_test)
print("...Done.")


Predictions on training set...
...Done.
Predictions on test set...
...Done.


In [49]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9740718051467115
R2 score on test set :  0.9364528193143951


In [50]:
scores = cross_val_score(regressor,X_train, Y_train, cv=10)
avg = scores.mean()
std = scores.std()
print('Cross-validated accuracy : {}\nstandard deviation : {}'.format(avg, std))

Cross-validated accuracy : 0.9427053017366035
standard deviation : 0.03984689793562844


Actually, the cross-validation shows that the score can typically vary by $\pm 0.04$ just by switching some examples between train and test set. We can conclude that there's **no significant difference** between the train score and the test score! The model has a good balance between bias and variance, which is as good news!

## OVERFITTING : Regularisation with Ridge & Lasso + GridSearch

In [55]:
# Perform grid search
print("Grid search...")
ridge = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100] # 0 corresponds to no regularization
}
gridsearch_r = GridSearchCV(ridge, param_grid = params) 
gridsearch_r.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_r.best_params_)
print("Best R2 score : ", gridsearch_r.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.01}
Best R2 score :  0.9459875694585662


In [56]:
# Perform grid search
print("Grid search...")
Lasso = linear_model.Lasso()
# Grid of values to be tested
params = {
    'alpha': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100] # 0 corresponds to no regularization
}
gridsearch_l = GridSearchCV(Lasso, param_grid = params)
gridsearch_l.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_l.best_params_)
print("Best R2 score : ", gridsearch_l.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 50}
Best R2 score :  0.9462863490580047


Ridge reduce the coefficient of features and Lasso reduce the number of features 
The Gridsearch is usefull to test different number for this hyper parameter and find the best. 

Our LinearRegression perfomed well but we can see that with a regulation the R2 augmented a bit. 🤓